In [159]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import *

<h1>Math Sanity Check</h1>

Hamiltonian in terms of Pauli Matrices: </br>
$H=\sum_{n=0}{c_nP_n}$

In [116]:
def HS(M1, M2):
    """Hilbert-Schmidt-Product of two matrices M1, M2"""
    return (np.dot(M1.conjugate().transpose(), M2)).trace()

def c2s(c):
    """Return a string representation of a complex number c"""
    if c == 0.0:
        return "0"
    if c.imag == 0:
        return "%g" % c.real
    elif c.real == 0:
        return "%gj" % c.imag
    else:
        return "%g+%gj" % (c.real, c.imag)

def decompose(H):
    """Decompose Hermitian 4x4 matrix H into Pauli matrices"""
    from numpy import kron
    sx = np.array([[0, 1],  [ 1, 0]], dtype=np.complex128)
    sy = np.array([[0, -1j],[1j, 0]], dtype=np.complex128)
    sz = np.array([[1, 0],  [0, -1]], dtype=np.complex128)
    id = np.array([[1, 0],  [ 0, 1]], dtype=np.complex128)
    S = [id, sx, sy, sz]
    labels = ['I', 'sigma_x', 'sigma_y', 'sigma_z']
    for i in range(4):
        for j in range(4):
            for k in range(4):
                label = labels[i] + ' \otimes ' + labels[j] + '\otimes' +labels[k]
                a_ij = (1/8) * HS(kron(kron(S[i], S[j]),S[k]), H)
                if a_ij != 0.0:
                    print("%s\t*\t( %s )" % (c2s(a_ij), label))


H = np.array(np.diag([0,0,0,1,0,0,0,1]), dtype=np.complex128)
decompose(H)
print(H)

0.25	*	( I \otimes I\otimesI )
-0.25	*	( I \otimes I\otimessigma_z )
-0.25	*	( I \otimes sigma_z\otimesI )
0.25	*	( I \otimes sigma_z\otimessigma_z )
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]]


In [ ]:
sx = np.array([[0, 1],  [ 1, 0]], dtype=np.complex128)
sy = np.array([[0, -1j],[1j, 0]], dtype=np.complex128)
sz = np.array([[1, 0],  [0, -1]], dtype=np.complex128)
id = np.array([[1, 0],  [ 0, 1]], dtype=np.complex128)

def multi_kron(*matrices):
    # Base case: If there are no matrices, return None
    if len(matrices) == 0:
        return None
    # Base case: If there is only one matrix, return it
    elif len(matrices) == 1:
        return matrices[0]
    else:
        # Compute the Kronecker product of the first two matrices
        result = np.kron(matrices[0], matrices[1])
        # Recursively compute the Kronecker product with the rest of the matrices
        return multi_kron(result, *matrices[2:])



In [271]:

import itertools
from qiskit import quantum_info





def decompose2(H):
    orig_mat = []
    def HS(M1, M2):
        """Hilbert-Schmidt-Product of two matrices M1, M2"""
        return (np.dot(M1.conjugate().transpose(), M2)).trace()
    def generate_combinations(N):
        ranges=[[0,3],]*int(np.log2(N))
        combinations = list(itertools.product(*[range(start, end+1) for start, end in ranges]))
        for i,j in enumerate(combinations):
            combinations[i] = list(j)
        return combinations
    """Decompose Hermitian 2^nx2^n matrix H into Pauli matrices"""
    from numpy import kron
    sx = np.array([[0, 1],  [ 1, 0]], dtype=np.complex128)
    sy = np.array([[0, -1j],[1j, 0]], dtype=np.complex128)
    sz = np.array([[1, 0],  [0, -1]], dtype=np.complex128)
    id = np.array([[1, 0],  [ 0, 1]], dtype=np.complex128)
    S = [id, sx, sy, sz]
    labels = ['I', 'X', 'Y', 'Z']
    id = np.array([[1, 0],  [ 0, 1]], dtype=np.complex128)
    S = [id, sx, sy, sz]
    labels = ['I', 'X', 'Y', 'Z']
    for i in generate_combinations(max(np.shape(H))):
        s_string = []
        l_string = []
        for j in i:
            s_string.append(S[j])
            l_string.append(labels[j])
        a_ij = (1/max(np.shape(H))) * HS(multi_kron(*s_string), H)
        if a_ij != 0.0:
            orig_mat.append(a_ij*quantum_info.Pauli(''.join(l_string)).to_matrix())
            print("%s\t*\t( %s )" % (c2s(a_ij), quantum_info.Pauli(''.join(l_string))))
    #return(sum(orig_mat))
    for i in generate_combinations(max(np.shape(H))):
        s_string = []
        l_string = []
        for j in i:
            s_string.append(S[j])
            l_string.append(labels[j])
        a_ij = (1/max(np.shape(H))) * HS(multi_kron(*s_string), H)
        if a_ij != 0.0:
            orig_mat.append(a_ij*quantum_info.Pauli(''.join(l_string)).to_matrix())
            print("%s\t*\t( %s )" % (c2s(a_ij), quantum_info.Pauli(''.join(l_string))))
    #return(sum(orig_mat))

H = np.array(np.diag([0,0.7,0,0,0,0.7,0,0]), dtype=np.complex128)
decompose2(H)

0.175	*	( III )
-0.175	*	( IIZ )
0.175	*	( IZI )
-0.175	*	( IZZ )
0.175	*	( III )
-0.175	*	( IIZ )
0.175	*	( IZI )
-0.175	*	( IZZ )


In [166]:
from qiskit import quantum_info

quantum_info.Pauli('III').to_matrix()


array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [273]:
import sys, math

def bMax(g, l, typ, pre):
    result = pre * g * 2 * l * math.sqrt(math.pi / 2 / (2*l+1))
    if (result > l * 2 * math.pi / (2*l+1) and typ=='c'):
        result = l * 2 * math.pi / (2*l+1)
    return result

def db(l, bMax):
    return(bMax / l)

def b(l, bMax, i):
    return(-bMax + i * db(l, bMax))

def intValues(l, i):
    result = np.zeros(3)
    for k in range(2, 0, -1):
        result[k] = int(i % (2**l))
        i = int(i / (2**l))
    result[0] = i
    return(result)


def Bop(l, bMax):
    result = np.zeros(((2**l), (2**l)))
    for i in range(2**l):
        result[i,i] = b(l, bMax, i)
    print(np.shape(result))
    return result

def B2op(l, bMax):
    result = np.zeros(((2**l), (2**l)))
    for i in range(2**l):
        result[i,i] = pow(b(l, bMax, i), 2)
    print(np.shape(result))
    return result

def B2opC(l, bMax):
    result = np.zeros((2**l, 2**l))
    for i in range(2*l):
        result[i,i] = 2. - 2. * math.cos(b(l, bMax, i))
    return result

def B2opAll(l, bMax):
    result = np.zeros((pow(2, 2*l), pow(2, 2*l)))
    for i in range(pow(2, 2*l)):
        ints = intValues(l, i)
        result[i,i] = pow(b(l, bMax, ints[0]) + b(l, bMax, ints[1]) + b(l, bMax, ints[2]), 2)
    return result

def B2opCAll(l, bMax):
    result = np.zeros((pow(2, l), pow(2, l)))
    for i in range(pow(2, l)):
        ints = intValues(l, i)
        result[i,i] = 2. - 2. * math.cos(b(l, bMax, ints[0]) + b(l, bMax, ints[1]) + b(l, bMax, ints[2]))
    return result


def opLattice1(pos, op):
    #print(op)
    size = np.shape(op)[0]
    id = np.identity(size)
    #print(op)
    #print(id)
    if (pos == 0):
        result = op
    else:
        result = id
    for i in range(1, 2):
        if (i == pos):
            result = np.kron(result, op)
        else:
            result = np.kron(result, id)
        #print(np.shape(result))
    return result

def opLattice2(pos1, pos2, op1, op2):
    size = np.shape(op1)[0]
    id = np.identity(size)
    if (pos1 == 0):
        result = op1
    elif (pos2 == 0):
        result = op2
    else:
        result = id
    for i in range(1, 1):
        if (i == pos1):
            result = np.kron(result, op1)
        elif (i == pos2):
            result = np.kron(result, op2)
        else:
            result = np.kron(result, id)
    return result

def opLatticeAll(op):
    result = op
    for i in range(2):
        result = np.kron(result, op)
    return result

def HB(g, l, bMax, typ):
    if (typ == 'c'):
        B2 = B2opC(l, bMax)
        B2All = B2opCAll(l, bMax)
    elif (typ == 'nc'):
        B2 = B2op(l, bMax)
        B2All = B2opAll(l, bMax)
    else:
        sys.exit("typ should be c for compact or nc for non-compact")
    print(np.shape(B2))
    return (opLattice1(0, B2) + opLattice1(1, B2) + opLattice1(2, B2) + B2All) / 2 / pow(g,2)

test = HB(1,3,4,'nc')
decompose2(test)

(8, 8)
(8, 8)
23.1667	*	( IIIIII )
1.33333	*	( IIIIIZ )
2.66667	*	( IIIIZI )
1.77778	*	( IIIIZZ )
5.33333	*	( IIIZII )
3.55556	*	( IIIZIZ )
7.11111	*	( IIIZZI )
8.88178e-16	*	( IIIZZZ )
1.33333	*	( IIZIII )
0.444444	*	( IIZIIZ )
0.888889	*	( IIZIZI )
-2.22045e-16	*	( IIZIZZ )
1.77778	*	( IIZZII )
2.22045e-16	*	( IIZZIZ )
-2.22045e-16	*	( IIZZZI )
-1.11022e-16	*	( IIZZZZ )
2.66667	*	( IZIIII )
0.888889	*	( IZIIIZ )
1.77778	*	( IZIIZI )
-3.33067e-16	*	( IZIIZZ )
3.55556	*	( IZIZII )
-3.33067e-16	*	( IZIZIZ )
2.22045e-16	*	( IZIZZI )
-1.11022e-16	*	( IZIZZZ )
1.77778	*	( IZZIII )
-2.22045e-16	*	( IZZIIZ )
-4.44089e-16	*	( IZZIZI )
-5.55112e-16	*	( IZZIZZ )
5.55112e-17	*	( IZZZII )
2.77556e-16	*	( IZZZIZ )
-1.66533e-16	*	( IZZZZI )
-8.32667e-16	*	( IZZZZZ )
5.33333	*	( ZIIIII )
1.77778	*	( ZIIIIZ )
3.55556	*	( ZIIIZI )
1.11022e-15	*	( ZIIIZZ )
7.11111	*	( ZIIZII )
1.11022e-15	*	( ZIIZIZ )
-4.44089e-16	*	( ZIIZZI )
-2.22045e-16	*	( ZIIZZZ )
3.55556	*	( ZIZIII )
4.44089e-16	*	( ZIZIIZ )
2.22